# T1090.001 - Internal Proxy
Adversaries may use an internal proxy to direct command and control traffic between two or more systems in a compromised environment. Many tools exist that enable traffic redirection through proxies or port redirection, including [HTRAN](https://attack.mitre.org/software/S0040), ZXProxy, and ZXPortMap. (Citation: Trend Micro APT Attack Tools) Adversaries use internal proxies to manage command and control communications inside a compromised environment, to reduce the number of simultaneous outbound network connections, to provide resiliency in the face of connection loss, or to ride over existing trusted communications paths between infected systems to avoid suspicion. Internal proxy connections may use common peer-to-peer (p2p) networking protocols, such as SMB, to better blend in with the environment.

By using a compromised internal system as a proxy, adversaries may conceal the true destination of C2 traffic while reducing the need for numerous connections to external systems.

## Atomic Tests

### Atomic Test #1 - Connection ProxyEnable traffic redirection.

Note that this test may conflict with pre-existing system configuration.
**Supported Platforms:** linux, macos#### Attack Commands: Run with `sh`
```sh
export http_proxy=127.0.0.1:8080
```

In [ ]:
Invoke-AtomicTest T1090.001 -TestNumbers 1

#### Cleanup: 
```sh
unset http_proxy
unset https_proxy
```

In [ ]:
Invoke-AtomicTest T1090.001 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Connection Proxy for macOS UIEnable traffic redirection on macOS UI (not terminal).
The test will modify and enable the "Web Proxy" and "Secure Web Proxy" settings  in System Preferences => Network => Advanced => Proxies for the specified network interface.

Note that this test may conflict with pre-existing system configuration.
**Supported Platforms:** macos#### Attack Commands: Run with `sh`
```sh
networksetup -setwebproxy Wi-Fi 127.0.0.1 8080
networksetup -setsecurewebproxy Wi-Fi 127.0.0.1 8080
```

In [ ]:
Invoke-AtomicTest T1090.001 -TestNumbers 2

#### Cleanup: 
```sh
networksetup -setwebproxystate Wi-Fi off
networksetup -setsecurewebproxystate Wi-Fi off  
```

In [ ]:
Invoke-AtomicTest T1090.001 -TestNumbers 2 -Cleanup

### Atomic Test #3 - portproxy reg keyAdds a registry key to set up a proxy on the endpoint at HKEY_LOCAL_MACHINE\SYSTEM\CurrentControlSet\Services\PortProxy\v4tov4
Upon execution there will be a new proxy entry in netsh
netsh interface portproxy show all
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
netsh interface portproxy add v4tov4 listenport=1337 connectport=1337 connectaddress=127.0.0.1```

In [ ]:
Invoke-AtomicTest T1090.001 -TestNumbers 3

#### Cleanup: 
```powershell
netsh interface portproxy delete v4tov4 listenport=1337 -ErrorAction Ignore | Out-Null```

In [ ]:
Invoke-AtomicTest T1090.001 -TestNumbers 3 -Cleanup

## Detection
Analyze network data for uncommon data flows between clients that should not or often do not communicate with one another. Processes utilizing the network that do not normally have network communication or have never been seen before are suspicious. Analyze packet contents to detect communications that do not follow the expected protocol behavior for the port that is being used.(Citation: University of Birmingham C2)